In [1]:
# import os
# import cv2
# import shutil

# def extract_frames_from_videos(video_folder, output_folder, fps=2):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
    
#     for category in ['drowsiness', 'not drowsiness']:
#         input_category_path = os.path.join(video_folder, category)
#         output_category_path = os.path.join(output_folder, category)
        
#         if not os.path.exists(output_category_path):
#             os.makedirs(output_category_path)
        
#         for video_file in os.listdir(input_category_path):
#             video_path = os.path.join(input_category_path, video_file)
#             if not video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
#                 continue
            
#             cap = cv2.VideoCapture(video_path)
#             frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
#             frame_interval = max(1, frame_rate // fps)
#             count = 0
#             frame_count = 0
            
#             while cap.isOpened():
#                 ret, frame = cap.read()
#                 if not ret:
#                     break
                
#                 if count % frame_interval == 0:
#                     image_name = f"{os.path.splitext(video_file)[0]}_frame{frame_count}.jpg"
#                     image_path = os.path.join(output_category_path, image_name)
#                     cv2.imwrite(image_path, frame)
#                     frame_count += 1
                
#                 count += 1
#             cap.release()
            

# def copy_images(source_folder, dest_folder):
#     if not os.path.exists(dest_folder):
#         os.makedirs(dest_folder)
    
#     for file in os.listdir(source_folder):
#         source_path = os.path.join(source_folder, file)
#         dest_path = os.path.join(dest_folder, file)
        
#         if os.path.isfile(source_path):
#             shutil.copy(source_path, dest_path)
            
# # video_folder = "/kaggle/input/sust-ddd/SUST Driver Drowsiness Dataset"
# # output_folder = "sust-ddd"
# # extract_frames_from_videos(video_folder, output_folder)

# copy_images("/kaggle/input/mrl-eye-dataset/mrleyedataset/Close-Eyes", "sust-ddd/drowsiness")
# copy_images("/kaggle/input/mrl-eye-dataset/mrleyedataset/Open-Eyes", "sust-ddd/not drowsiness")

# copy_images("/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/Drowsy", "sust-ddd/drowsiness")
# copy_images("/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/Non Drowsy", "sust-ddd/not drowsiness")

# print("Completed1")

In [2]:
# import os
# import shutil
# import random

# def split_dataset(input_folder, output_folder, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
    
#     for category in ['drowsiness', 'not drowsiness']:
#         category_path = os.path.join(input_folder, category)
#         images = os.listdir(category_path)
#         random.shuffle(images)
        
#         train_split = int(len(images) * train_ratio)
#         val_split = int(len(images) * (train_ratio + val_ratio))
        
#         sets = {'train': images[:train_split], 'val': images[train_split:val_split], 'test': images[val_split:]}
        
#         for split, image_list in sets.items():
#             split_path = os.path.join(output_folder, split, category)
#             if not os.path.exists(split_path):
#                 os.makedirs(split_path)
            
#             for image in image_list:
#                 src_path = os.path.join(category_path, image)
#                 dest_path = os.path.join(split_path, image)
#                 shutil.copy(src_path, dest_path)
                
# input_folder = "sust-ddd"
# output_folder = "sust-ddd-split"
# split_dataset(input_folder, output_folder)
# print("Completed1")
# print("Training Starts")

In [3]:
# !pip install facenet_pytorch
# !pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [4]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.6 which is incompatible.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.1

In [5]:
# import os
# import cv2
# import mediapipe as mp
# import numpy as np
# from tqdm import tqdm

# # Define paths
# input_dir = "/kaggle/input/sust-ddd/SUST Driver Drowsiness Dataset"
# output_dir = "/kaggle/working/extracted_faces"

# # Ensure output directories exist
# os.makedirs(output_dir, exist_ok=True)
# os.makedirs(f"{output_dir}/drowsiness", exist_ok=True)
# os.makedirs(f"{output_dir}/non-drowsiness", exist_ok=True)

# # Initialize MediaPipe face detector
# mp_face_detection = mp.solutions.face_detection
# face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# # Function to extract faces with margin
# def extract_faces(video_path, output_folder, margin=50, frame_skip=5):
#     cap = cv2.VideoCapture(video_path)
#     frame_count = 0
#     saved_count = 0
    
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Process every nth frame
#         if frame_count % frame_skip == 0:
#             img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             results = face_detection.process(img_rgb)
            
#             if results.detections:
#                 for detection in results.detections:
#                     bboxC = detection.location_data.relative_bounding_box
#                     h, w, _ = frame.shape
#                     x1 = int(bboxC.xmin * w) - margin
#                     y1 = int(bboxC.ymin * h) - margin
#                     x2 = int((bboxC.xmin + bboxC.width) * w) + margin
#                     y2 = int((bboxC.ymin + bboxC.height) * h) + margin

#                     # Ensure bounding box stays within frame
#                     x1, y1 = max(0, x1), max(0, y1)
#                     x2, y2 = min(w, x2), min(h, y2)

#                     face = frame[y1:y2, x1:x2]
                    
#                     if face.size > 0:
#                         save_path = os.path.join(output_folder, f"frame_{saved_count:04d}.jpg")
#                         cv2.imwrite(save_path, face)
#                         saved_count += 1
        
#         frame_count += 1

#     cap.release()

# # Process each video in both categories
# for category in ["drowsiness", "not drowsiness"]:
#     category_path = os.path.join(input_dir, category)
#     output_folder = os.path.join(output_dir, category.replace(" ", "-"))  # Replace spaces
    
#     for video_file in tqdm(os.listdir(category_path), desc=f"Processing {category} videos"):
#         video_path = os.path.join(category_path, video_file)
#         extract_faces(video_path, output_folder)

# print("Face extraction complete!")


In [6]:
# import os
# import cv2
# import mediapipe as mp
# import numpy as np
# from tqdm import tqdm

# # Define paths
# input_dir = "/kaggle/input/sust-ddd/SUST Driver Drowsiness Dataset"
# output_dir = "/kaggle/working/extracted_faces"

# # Ensure output directories exist
# os.makedirs(output_dir, exist_ok=True)
# os.makedirs(f"{output_dir}/drowsiness", exist_ok=True)
# os.makedirs(f"{output_dir}/non-drowsiness", exist_ok=True)

# # Initialize MediaPipe face detector
# mp_face_detection = mp.solutions.face_detection
# face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# # Function to extract faces with larger coverage and save in respective folders
# def extract_faces(video_path, output_folder, frame_skip=5):
#     cap = cv2.VideoCapture(video_path)
#     frame_count = 0
#     saved_count = 0
#     video_name = os.path.splitext(os.path.basename(video_path))[0]  # Extract video name
    
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Process every nth frame
#         if frame_count % frame_skip == 0:
#             img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             results = face_detection.process(img_rgb)
            
#             if results.detections:
#                 for detection in results.detections:
#                     bboxC = detection.location_data.relative_bounding_box
#                     h, w, _ = frame.shape
#                     x_min, y_min = int(bboxC.xmin * w), int(bboxC.ymin * h)
#                     x_max, y_max = int((bboxC.xmin + bboxC.width) * w), int((bboxC.ymin + bboxC.height) * h)

#                     # Calculate dynamic margin (30% of face size)
#                     margin_x = int(0.3 * (x_max - x_min))
#                     margin_y = int(0.3 * (y_max - y_min))

#                     # Apply margin
#                     x1, y1 = max(0, x_min - margin_x), max(0, y_min - margin_y)
#                     x2, y2 = min(w, x_max + margin_x), min(h, y_max + margin_y)

#                     face = frame[y1:y2, x1:x2]
                    
#                     if face.size > 0:
#                         save_path = os.path.join(output_folder, f"{video_name}_frame_{saved_count:04d}.jpg")
#                         cv2.imwrite(save_path, face)
#                         saved_count += 1
        
#         frame_count += 1

#     cap.release()

# # Process each video in both categories and save correctly
# for category in ["drowsiness", "not drowsiness"]:
#     category_path = os.path.join(input_dir, category)
#     output_folder = os.path.join(output_dir, category.replace(" ", "-"))  # Ensure folder name is correct
    
#     for video_file in tqdm(os.listdir(category_path), desc=f"Processing {category} videos"):
#         video_path = os.path.join(category_path, video_file)
#         extract_faces(video_path, output_folder)

# print("Face extraction complete!")


In [7]:
# import os
# import shutil
# import random

# # Define paths
# input_dir = "/kaggle/working/extracted_faces"
# output_dir = "/kaggle/working/split_data"

# # Train-test split ratio
# train_ratio = 0.8  

# # Create train and test directories
# for split in ["train", "test"]:
#     for category in ["drowsiness", "non-drowsiness"]:
#         os.makedirs(os.path.join(output_dir, split, category), exist_ok=True)

# # Function to split data
# def split_data():
#     for category in ["drowsiness", "non-drowsiness"]:
#         image_dir = os.path.join(input_dir, category)
#         images = os.listdir(image_dir)

#         # Shuffle images randomly
#         random.shuffle(images)

#         # Split into train and test
#         split_idx = int(len(images) * train_ratio)
#         train_images = images[:split_idx]
#         test_images = images[split_idx:]

#         # Move files to respective folders
#         for img in train_images:
#             src = os.path.join(image_dir, img)
#             dst = os.path.join(output_dir, "train", category, img)
#             shutil.move(src, dst)  # Use move instead of copy

#         for img in test_images:
#             src = os.path.join(image_dir, img)
#             dst = os.path.join(output_dir, "test", category, img)
#             shutil.move(src, dst)  # Use move instead of copy

# # Run the split function
# split_data()
# print("Data split complete!")


In [8]:
!pip install mediapipe

In [9]:
import os
import cv2
import mediapipe as mp
import numpy as np
import random
from tqdm import tqdm
import shutil

# Define Paths
input_dir = "/kaggle/input/sust-ddd/SUST Driver Drowsiness Dataset"
output_dir = "/kaggle/working/split_data"

# Train-test split ratio
train_ratio = 0.8  

# Create train and test directories
for split in ["train", "test"]:
    for category in ["drowsiness", "not-drowsiness"]:
        os.makedirs(os.path.join(output_dir, split, category), exist_ok=True)

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Function to Extract Faces & Save Directly into Train/Test Folders
def extract_and_save_faces(video_path, category):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_faces = []
    video_name = os.path.splitext(os.path.basename(video_path))[0]  # Extract video name

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process every 5th frame for efficiency
        if frame_count % 5 == 0:
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(img_rgb)
            
            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    h, w, _ = frame.shape
                    x_min, y_min = int(bboxC.xmin * w), int(bboxC.ymin * h)
                    x_max, y_max = int((bboxC.xmin + bboxC.width) * w), int((bboxC.ymin + bboxC.height) * h)

                    # Add a margin (30% of face size)
                    margin_x = int(0.3 * (x_max - x_min))
                    margin_y = int(0.3 * (y_max - y_min))

                    # Apply margin while keeping within image bounds
                    x1, y1 = max(0, x_min - margin_x), max(0, y_min - margin_y)
                    x2, y2 = min(w, x_max + margin_x), min(h, y_max + margin_y)

                    face = frame[y1:y2, x1:x2]

                    if face.size > 0:
                        saved_faces.append(face)

        frame_count += 1

    cap.release()

    # Shuffle and split into train/test
    random.shuffle(saved_faces)
    split_idx = int(len(saved_faces) * train_ratio)
    train_faces = saved_faces[:split_idx]
    test_faces = saved_faces[split_idx:]

    # Save faces
    for i, face in enumerate(train_faces):
        save_path = os.path.join(output_dir, "train", category, f"{video_name}_frame_{i:04d}.jpg")
        cv2.imwrite(save_path, face)

    for i, face in enumerate(test_faces):
        save_path = os.path.join(output_dir, "test", category, f"{video_name}_frame_{i:04d}.jpg")
        cv2.imwrite(save_path, face)

# Process each video in both categories
for category in ["drowsiness", "not drowsiness"]:
    category_path = os.path.join(input_dir, category)
    save_category = category.replace(" ", "-")  # Ensure valid folder naming

    for video_file in tqdm(os.listdir(category_path), desc=f"Processing {category} videos"):
        video_path = os.path.join(category_path, video_file)
        extract_and_save_faces(video_path, save_category)

print("Face extraction and train-test split complete!")


Processing not drowsiness videos: 100%|██████████| 1099/1099 [18:12<00:00,  1.01it/s]

Face extraction and train-test split complete!


In [10]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define Paths
data_dir = "/kaggle/working/split_data"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Hyperparameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.0001

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    zoom_range=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load Train & Test Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Load Pretrained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Custom Head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary Classification (Drowsy vs Non-Drowsy)

# Create Model
model = Model(inputs=base_model.input, outputs=x)

# Compile Model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train Model
history = model.fit(train_generator, validation_data=test_generator, epochs=EPOCHS)

# Save Model as .h5
model.save("/kaggle/working/mobilenetv2_drowsiness.h5")
print("Model saved successfully!")


Found 98774 images belonging to 2 classes.
Found 24795 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


386/386 ━━━━━━━━━━━━━━━━━━━━ 1164s 3s/step - accuracy: 0.6725 - loss: 0.6011 - val_accuracy: 0.7877 - val_loss: 0.4786
Epoch 2/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1129s 3s/step - accuracy: 0.7787 - loss: 0.4806 - val_accuracy: 0.7981 - val_loss: 0.4549
Epoch 3/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1128s 3s/step - accuracy: 0.7952 - loss: 0.4522 - val_accuracy: 0.8091 - val_loss: 0.4302
Epoch 4/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1138s 3s/step - accuracy: 0.8033 - loss: 0.4321 - val_accuracy: 0.8139 - val_loss: 0.4169
Epoch 5/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1145s 3s/step - accuracy: 0.8083 - loss: 0.4204 - val_accuracy: 0.8186 - val_loss: 0.4074
Epoch 6/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1129s 3s/step - accuracy: 0.8134 - loss: 0.4108 - val_accuracy: 0.8196 - val_loss: 0.4015
Epoch 7/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1128s 3s/step - accuracy: 0.8148 - loss: 0.4053 - val_accuracy: 0.8172 - val_loss: 0.4040
Epoch 8/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1144s 3s/step - accuracy: 0.8208 - loss: 0.3976 - val_accurac

In [11]:
!pip install onnx tf2onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
mediapipe 0.10.20 requires protobuf<5,>=4.25.3, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [12]:
import tf2onnx
import tensorflow as tf

# Define the input signature. This should match your model's input shape.
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)

# Specify the output file path for the ONNX model.
output_path = "/kaggle/working/mobilenetv2_drowsiness.onnx"

# Convert the Keras model to ONNX.
model_proto, _ = tf2onnx.convert.from_keras(
    model,         # your trained model
    input_signature=spec,
    opset=13,      # You can adjust the ONNX opset version if needed.
    output_path=output_path
)

print("Model successfully saved in ONNX format at", output_path)


Model successfully saved in ONNX format at /kaggle/working/mobilenetv2_drowsiness.onnx


In [13]:
# import os
# import torch
# import torchvision
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import datasets, transforms, models
# from torch.utils.data import DataLoader

# # Paths
# data_dir = "/kaggle/working/split_data"
# train_dir = os.path.join(data_dir, "train")
# test_dir = os.path.join(data_dir, "test")

# # Hyperparameters
# batch_size = 64
# num_epochs = 20
# learning_rate = 0.001
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Data Augmentation & Normalization
# transform = {
#     "train": transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(10),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     "test": transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])
# }

# # Load Datasets
# train_dataset = datasets.ImageFolder(train_dir, transform=transform["train"])
# test_dataset = datasets.ImageFolder(test_dir, transform=transform["test"])

# # Dataloaders
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# # Load Pretrained MobileNetV2
# model = models.mobilenet_v2(pretrained=True)

# # Modify classifier for 2 classes (Drowsy & Non-Drowsy)
# model.classifier[1] = nn.Linear(model.last_channel, 2)

# # Move model to GPU if available
# model = model.to(device)

# # Loss and Optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Training Function
# def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
#     model.train()
    
#     for epoch in range(num_epochs):
#         running_loss = 0.0
#         correct, total = 0, 0
        
#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)
            
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
            
#             running_loss += loss.item()
#             _, preds = torch.max(outputs, 1)
#             correct += (preds == labels).sum().item()
#             total += labels.size(0)

#         epoch_loss = running_loss / len(train_loader)
#         epoch_acc = correct / total * 100
#         print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
    
#     print("Training Complete!")

# # Run Training
# train_model(model, train_loader, criterion, optimizer, num_epochs)

# # Save the Model
# torch.save(model.state_dict(), "/kaggle/working/mobilenetv2_drowsiness.pth")
# print("Model saved successfully!")


In [14]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import os
# import matplotlib.pyplot as plt

# # Define paths
# data_dir = "sust-ddd-split"
# img_size = (224, 224)
# batch_size = 128

# # Data augmentation and preprocessing
# datagen = ImageDataGenerator(
#     rescale=1.0/255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2
# )

# train_data = datagen.flow_from_directory(
#     os.path.join(data_dir, "train"),
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='binary'
# )

# val_data = datagen.flow_from_directory(
#     os.path.join(data_dir, "val"),
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='binary'
# )

# # Pretrained Model - MobileNetV2
# base_model = keras.applications.MobileNetV2(input_shape=(*img_size, 3), include_top=False, weights="imagenet")
# base_model.trainable = False

# model = keras.Sequential([
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(128, activation="relu"),
#     layers.Dropout(0.3),
#     layers.Dense(1, activation="sigmoid")
# ])

# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
#               loss="binary_crossentropy",
#               metrics=["accuracy"])

# # Train the model
# history = model.fit(train_data, validation_data=val_data, epochs=10)

# # Plot loss and accuracy
# plt.figure(figsize=(12, 5))

# plt.subplot(1, 2, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.title('Loss over Epochs')

# plt.subplot(1, 2, 2)
# plt.plot(history.history['accuracy'], label='Train Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.title('Accuracy over Epochs')

# plt.show()

In [15]:
# model.save("model.h5")
# import tensorflow as tf

# # # Convert the model to TFLite
# # converter = tf.lite.TFLiteConverter.from_keras_model(model)
# # tflite_model = converter.convert()

# # # Save the TFLite model
# # with open("model.tflite", "wb") as f:
# #     f.write(tflite_model)


# # import tf2onnx

# # # Convert to ONNX format
# # onnx_model, _ = tf2onnx.convert.from_keras(model, output_path="model.onnx")

# # !pip install tensorflowjs
# # import tensorflowjs as tfjs

# # # Convert to TF.js format
# # tfjs.converters.save_keras_model(model, "tfjs_model")


In [16]:
# !pip install ultralytics

In [17]:
# from ultralytics import YOLO

# yolo_model = YOLO("yolov8n-cls.pt")
# yolo_model.train(data=data_dir, epochs=10, imgsz=224, batch=32)
